In [9]:
%%time
from functools import partial
from collections import defaultdict, namedtuple
import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from itertools import product
import time

from video699.screen.semantic_segmentation.fastai_detector import *
from video699.screen.semantic_segmentation.common import *
from video699.screen.semantic_segmentation.postprocessing import *
from video699.screen.semantic_segmentation.evaluation import *
from video699.screen.annotated import *

Split = namedtuple('Split', 'train valid')
all_lectures = [video.filename for video in ALL_VIDEOS]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 207 µs


In [10]:
actual_detector = AnnotatedSampledVideoScreenDetector()

In [11]:
all_frames = [frame for video in ALL_VIDEOS for frame in video]

In [12]:
all_frames_grouped_by_videos = {video.filename: [frame for frame in video] for video in ALL_VIDEOS}

In [14]:
kf = KFold(n_splits=5, shuffle=True, random_state=123)
outer_splits = {}
for i, split in tqdm(enumerate(kf.split(all_lectures))):
    other_lectures = [all_lectures[index] for index in split[0]]
    test_lectures = [all_lectures[index] for index in split[1]]
    outer_splits[i] = Split(train=other_lectures, valid=test_lectures)
    test_frames = []
    for lecture in test_lectures:
        test_frames.extend(all_frames_grouped_by_videos[lecture])
    test_screens = sum([len(actual_detector.detect(frame)) for frame in test_frames])
    print(test_screens)

5it [00:00, 1127.38it/s]

171
127
122
148
115


In [15]:
number_of_screens_in_test_split = [171, 127, 122, 148, 115]

## Cross validation summary

### Image-wise

In [25]:
df_cv = pd.read_csv("cross_validation_results_image_wise.csv")
iou_score = df_cv['iou'].mean()
print(f"IoU: {iou_score:.2f}")
df_cv

IoU: 0.97


,resize_factor,frozen_epochs,unfrozen_epochs,base_lower_bound,erosion_dilation_kernel_size,ratio_split_lower_bound,iou,wrong_count
0,2.0,7.0,4.0,15.0,20.0,0.4,0.973843,0.0
1,2.0,7.0,4.0,5.0,20.0,0.4,0.969167,1.0
2,2.0,7.0,4.0,15.0,20.0,0.4,0.971859,1.0
3,2.0,4.0,7.0,15.0,20.0,0.4,0.968294,1.0
4,2.0,7.0,4.0,10.0,20.0,0.4,0.972639,0.0


### Lecture-wise

In [26]:
df_cv = pd.read_csv("cross_validation_results_lecture_wise.csv")
iou_score = df_cv['iou'].mean()
print(f"IoU: {iou_score:.2f}")
df_cv

FileNotFoundError: [Errno 2] File cross_validation_results_lecture_wise.csv does not exist: 'cross_validation_results_lecture_wise.csv'

### Lecture peeking

In [27]:
df_cv = pd.read_csv("cross_validation_results_peeking_wise.csv")
iou_score = df_cv['iou'].mean()
print(f"IoU: {iou_score:.2f}")
df_cv

IoU: 0.96


,resize_factor,frozen_epochs,unfrozen_epochs,base_lower_bound,erosion_dilation_kernel_size,ratio_split_lower_bound,iou,wrong_count
0,2.0,7.0,7.0,15.0,20.0,0.4,0.966933,0.0
1,2.0,7.0,7.0,5.0,20.0,0.4,0.971831,3.0
2,2.0,4.0,7.0,5.0,50.0,0.4,0.962015,4.0
3,2.0,7.0,7.0,5.0,80.0,0.4,0.971366,0.0
4,2.0,7.0,4.0,15.0,20.0,0.4,0.942070,1.0


### Smaller lecture

In [28]:
df_cv = pd.read_csv("cross_validation_results_lecture_wise-smaller.csv")
iou_score = df_cv['iou'].mean()
print(f"IoU: {iou_score:.2f}")
df_cv

IoU: 0.91


,resize_factor,frozen_epochs,unfrozen_epochs,base_lower_bound,erosion_dilation_kernel_size,ratio_split_lower_bound,iou,wrong_count
0,8.0,4.0,7.0,15.0,20.0,0.4,0.926394,0.0
1,8.0,4.0,7.0,15.0,20.0,0.4,0.915838,20.0
2,8.0,4.0,4.0,5.0,50.0,0.5,0.929392,3.0
3,8.0,7.0,4.0,15.0,20.0,0.4,0.907802,1.0
4,8.0,7.0,7.0,15.0,20.0,0.4,0.894929,15.0


### Bachelor thesis

In [29]:
image = "cross_validation_results_image_wise.csv"
lecture = "cross_validation_results_lecture_wise.csv"
peeking = "cross_validation_results_peeking_wise.csv"
smaller = "cross_validation_results_lecture_smaller.csv"

In [31]:
df = pd.read_csv(smaller)

In [32]:
df

,resize_factor,frozen_epochs,unfrozen_epochs,base_lower_bound,erosion_dilation_kernel_size,ratio_split_lower_bound,iou,wrong_count
0,8.0,4.0,7.0,15.0,20.0,0.4,0.926394,0.0
1,8.0,4.0,7.0,15.0,20.0,0.4,0.915838,20.0
2,8.0,4.0,4.0,5.0,50.0,0.5,0.929392,3.0
3,8.0,7.0,4.0,15.0,20.0,0.4,0.907802,1.0
4,8.0,7.0,7.0,15.0,20.0,0.4,0.894929,15.0


In [33]:
print(round(df['iou'].mean(), 3))
print(" & ".join(list(map(str,list(df['iou'].round(3))))))

wrong_counts = df['wrong_count'].astype('int')
wrong_counts = ((wrong_counts / number_of_screens_in_test_split) * 100).round(2)
print(" & ".join(map(str, list(wrong_counts))))
print(wrong_counts.mean())

0.915
0.926 & 0.916 & 0.929 & 0.908 & 0.895
0.0 & 15.75 & 2.46 & 0.68 & 13.04
6.386
